In [1]:
import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt
import seaborn as sns

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')

from nltk import sent_tokenize
from gensim.utils import simple_preprocess
nltk.download('punkt')

from sklearn.preprocessing import StandardScaler
import pickle
from sklearn import preprocessing

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
def onehot(ft,dataset):
    label = [x for x in dataset[ft].value_counts().head(10).index]
    for i in label:
        dataset[i] = np.where(dataset[ft] ==i,1,0)

def test(file):

    data = pd.read_csv(file,usecols = ['Summary','Issue Type','Status','Assignee','Priority','Created'])
    i = data[(data['Issue Type'] != 'Bug')].index
    data = data.drop(i,axis=0)

    df = pd.DataFrame(columns = ['Summary','Majorblocks','Assignee'])
    df.Summary = data['Summary']
    df.Assignee = data['Assignee']


    label_encoder = preprocessing.LabelEncoder()
    data['Status'] = label_encoder.fit_transform(data['Status'])
    data['Priority'] = label_encoder.fit_transform(data['Priority'])
    onehot('Assignee',data)

    data['Date'] = pd.to_datetime(data['Created']).dt.date
    data['Creatd_Time'] = pd.to_datetime(data['Created']).dt.time

    data["created_day"] = pd.to_datetime(data.Date, format="%Y-%m-%d").dt.day
    data["created_month"] = pd.to_datetime(data.Date, format="%Y-%m-%d").dt.month

    ind=data['Date'].index
    ind = list(ind)
    k=0
    index = []
    for i in data['Date'].values:
        if dt.strptime(str(i), "%Y-%m-%d")  >  dt.strptime("2022-09-27", "%Y-%m-%d"):
            # if (dt.strptime(str(j), "%H:%M:%S") > dt.strptime("20:41:00", "%H:%M:%S")):
            index.append(ind[k])
            k=k+1
    print(index)

    df['Date'] = data['Date']
    df['Time'] = data['Creatd_Time']
    

    data = data.drop(['Date','Created','Issue Type','Assignee','Creatd_Time'],axis=1)

   

    data['Summary'] = data['Summary'].apply(lambda x : re.sub('[^a-zA-Z0-9]',' ',x))
    corpus1 = []
    index1 = list(data.index)
    i=0
    lemmatizer=WordNetLemmatizer()
    for j in range(0,len(data.Summary)):
        title = data.Summary[index1[i]]
        title = title.lower()
        title = title.split()
        title = [lemmatizer.lemmatize(word) for word in title if not word in stopwords.words('english')]
        title = ' '.join(title)
        corpus1.append(title)
        i=i+1

    from sklearn.feature_extraction.text import TfidfVectorizer
    tv = TfidfVectorizer(max_features=2500)
    X = tv.fit_transform(corpus1).toarray()
    data.Summary = X
    
    

    sc = StandardScaler()
    x = data
    x = sc.fit_transform(x)


    model1 = open("RFS.pkl",'rb')
    #model2 = open("RFH.pkl",'rb')

    model1_RFS = pickle.load(model1)

    pred1 = model1_RFS.predict(x[index])

    j=0
    for i in pred1:
        df['Majorblocks'][index[j]] = i
        j=j+1
    
    df.to_csv('res_maj.csv')
    print(df.head(20))


    

    #print(pred1)
    #print(pred2)

In [7]:
test("/content/JIRA (4).csv")

[0, 5, 8, 12, 13, 18, 19, 21, 22, 27, 29, 31, 34, 36, 43, 45, 46, 48, 49, 50, 51, 57, 58, 64, 67, 71, 79, 82, 83, 85, 86, 87, 88, 89, 90, 91, 99, 102, 104, 106, 107, 110, 111, 112, 113, 116]
                                              Summary  \
0   [TGV] performance issue for mask producing com...   
5   [TGV]:: COSIM VS RTL VL/VTYPE MISMATCH incase ...   
8                        Dhrystone Performance in HB    
12                [TGV] Milestone 13 L2 tests failing   
13  [TGV]: SdIssQ Entry must have issued to be val...   
18  [TGV] Fix LSIssQ to handle multiple STBPipe br...   
19  [TGV] Fix LsIssQ assert "vagu require store da...   
21  [LSU FV]: PTW with DTLB MMIO is sending RH sou...   
22  [LSU FV]: Probe request unable to read L1DC du...   
27  [LSU FV]: PTW with DTLB MMIO Response is viola...   
29  [TGV] LsIssQEntry_assert : vagu require store ...   
31         [TGV] 32-bit unmasked strided load hanging   
34  Assertion Failure: unrolled load tail should f...   
36  [LSU FV

In [4]:
from datetime import datetime as dt